In [1]:
%matplotlib inline
import datetime
import calendar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from scipy.stats import randint 
from scipy.stats import uniform
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

c:\users\stijn\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
dataset = pd.read_csv('fake.csv')
dataset.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [3]:
dataset = pd.read_csv('fake.csv')
dd = dataset.drop('uuid', axis=1)
dd = dd.drop('ord_in_thread', axis=1)
dd = dd.drop('published', axis=1)
dd = dd.drop('language', axis=1)
dd = dd.drop('crawled', axis=1)
dd = dd.drop('site_url', axis=1)
dd = dd.drop('country', axis=1)
dd = dd.drop('domain_rank', axis=1)
dd = dd.drop('thread_title', axis=1)
dd = dd.drop('spam_score', axis=1)
dd = dd.drop('replies_count', axis=1)
dd = dd.drop('participants_count', axis=1)
dd = dd.drop('type', axis=1)
dd = dd.dropna()
dd.drop_duplicates(keep="first",inplace=True)
dd.drop_duplicates("title",inplace=True)
dd.shape

(6411, 7)

In [10]:
dd = dd.sort_values(by=['likes'], ascending=False)

In [11]:
dd.head()

,author,title,text,main_img_url,likes,comments,shares
8211,RT,Nat Geo’s iconic ‘Afghan Girl’ arrested for fa...,Nat Geo’s iconic ‘Afghan Girl’ arrested for fa...,https://img.rt.com/files/2016.10/article/5810e...,988,0,988
8256,RT на русском,Мечтатели из Пентагона: как армия США впустую ...,"Короткая ссылка 27 октября 2016, 00:10 Замести...",https://rus.rt.com/russian/images/2016.10/arti...,982,0,982
7243,Reuters,Clinton campaign urges FBI to detail new devel...,By Reuters 6:34 pm Democrat Hillary Clinton'...,http://15130-presscdn-0-89.pagely.netdna-cdn.c...,980,0,980
8202,RT,AT&T sold access to customer data to law enfor...,AT&T sold access to customer data to law enfor...,https://img.rt.com/files/2016.10/article/5810d...,974,0,974
786,Amando Flavio,NFL Warns Clenbuterol Level So High in Beef th...,The National Football League (NFL) in the Unit...,http://anonhq.com/wp-content/uploads/2016/10/s...,967,0,967


In [26]:
print(dd.iloc[0].title)

Nat Geo’s iconic ‘Afghan Girl’ arrested for false documents in Pakistan


In [28]:
dd = dd.sort_values(by=['comments'], ascending=False)
dd.head()

,author,title,text,main_img_url,likes,comments,shares
1769,John Falkenberg,Trump Haters Boycott Ivanka’s Clothing Line Be...,"“That is what ‘Trump’ means to me,” she said. ...",http://static.conservativetribune.com/wp-conte...,16,30,16
1767,V Saxena,PHOTO: Game Camera Catches Glimpse Of Possible...,Hillary Camp Caught on Camera Telling Tiny Cro...,http://static.conservativetribune.com/wp-conte...,903,26,903
1826,Wilmot Proviso,NBA Team Cancels Anthem Singer After She Tries...,“I was never given any kind of dress code. I w...,http://static.conservativetribune.com/wp-conte...,822,17,822
1760,Davis,WATCH: Gingrich Accuses Megyn Kelly Of Being “...,Hillary Howls in Laughter About Radical Muslim...,http://static.conservativetribune.com/wp-conte...,682,17,682
9765,The Free Thought Project,5 Shocking Facts About You That Anyone Can Fin...,Home / Foreign Affairs / 5 Shocking Facts Abou...,http://thefreethoughtproject.com/wp-content/up...,95,15,95


In [29]:
dd = dd.sort_values(by=['shares'], ascending=False)
dd.head()

,author,title,text,main_img_url,likes,comments,shares
8211,RT,Nat Geo’s iconic ‘Afghan Girl’ arrested for fa...,Nat Geo’s iconic ‘Afghan Girl’ arrested for fa...,https://img.rt.com/files/2016.10/article/5810e...,988,0,988
8256,RT на русском,Мечтатели из Пентагона: как армия США впустую ...,"Короткая ссылка 27 октября 2016, 00:10 Замести...",https://rus.rt.com/russian/images/2016.10/arti...,982,0,982
7243,Reuters,Clinton campaign urges FBI to detail new devel...,By Reuters 6:34 pm Democrat Hillary Clinton'...,http://15130-presscdn-0-89.pagely.netdna-cdn.c...,980,0,980
8202,RT,AT&T sold access to customer data to law enfor...,AT&T sold access to customer data to law enfor...,https://img.rt.com/files/2016.10/article/5810d...,974,0,974
9757,doucyet,Comment on There Has To Be a Cheaper Way To Fi...,Watch: Aerial Footage Catches Man Stopping at ...,http://thefreethoughtproject.com/wp-content/up...,967,0,967


In [30]:
dd.iloc[9]

author                                                Vicki Batts
title           Nestle to launch new non-GMO products ... How ...
text            Nestle to launch new non-GMO products ... How ...
main_img_url    http://www.naturalnews.com/gallery/640/GMOs/No...
likes                                                         944
comments                                                        0
shares                                                        944
Name: 6424, dtype: object